In [1]:
!pip install pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.6 MB/s eta 0:00:00


In [7]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from sklearn.preprocessing import MinMaxScaler
np.random.seed(46)
path = 'C:/Users/aless/OneDrive - Università degli Studi di Catania/tesi/dataset/final_df.csv'

In [9]:
df = pd.read_csv(path)
protocol_states = ['Base', 'TSST']
df = df[df['labels'].isin(protocol_states)]
mapping_dict = {'Base': 0,
                'TSST': 1}
df['labels'] = df['labels'].map(mapping_dict)
ID = df['ID'].reset_index(drop=True) #store id column for later usage
df.drop(['Time'], inplace=True, axis = 1)
df.head()

,ID,hr,temp,labels
420,0,75.563333,35.447233,0
421,0,75.570000,35.549821,0
422,0,75.573333,35.449041,0
423,0,75.593333,35.543187,0
424,0,75.643333,35.443003,0


In [11]:
scaler = MinMaxScaler()
columns_to_scale = ['hr', 'temp']
scaled_df = df.copy(deep=True)
scaled_df[['hr', 'temp']] = scaler.fit_transform(scaled_df[['hr', 'temp']])
scaled_df.head()

,ID,hr,temp,labels
420,0,0.385214,0.931318,0
421,0,0.385324,0.944449,0
422,0,0.385379,0.931549,0
423,0,0.385708,0.943600,0
424,0,0.386531,0.930776,0


In [13]:
output_folder = 'C:/Users/aless/OneDrive - Università degli Studi di Catania/tesi/dataset/gaf_imgs'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def apply_gaf_transformation(data_array, filename):
    transformer = GramianAngularField(image_size=250, sample_range=(0, 1))
    img = transformer.fit_transform(data_array)

    gaf = np.concatenate((img, np.zeros((1,250,250))), axis=0)

    gaf_img = (gaf * 255).astype(np.uint8)
    gaf_img = np.transpose(gaf_img, (1,2,0))
    pil_img = Image.fromarray(gaf_img)
    pil_img.save(filename)

In [14]:
for i in scaled_df['ID'].unique():
    for label in [0, 1]:
        df = scaled_df[(scaled_df['ID'] == i) & (scaled_df['labels'] == label)]
        array = df[['hr', 'temp']].values
        array = np.transpose(array)

        label_folder = os.path.join(output_folder, f'label_{label}')
        if not os.path.exists(label_folder):
            os.makedirs(label_folder)

        filename = os.path.join(label_folder, f'img_{i}.jpg')

        apply_gaf_transformation(array, filename)